In [1]:
!pip install cdsapi

In [2]:

import os, getpass, textwrap, pathlib


cfg = textwrap.dedent(f"""\
url: https://cds.climate.copernicus.eu/api
key: 55a51e6d-554d-46e6-8743-c8f5f4a98f9b
""")

path = pathlib.Path("~/.cdsapirc").expanduser()
path.write_text(cfg)
# 收紧权限（Linux 600）
!chmod 600 ~/.cdsapirc

print("~/.cdsapirc 写入完成")


~/.cdsapirc 写入完成


In [3]:
import cdsapi
c = cdsapi.Client()
print("CDS client OK")


2025-09-28 10:16:07,307 INFO [2025-09-03T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
INFO:ecmwf.datastores.legacy_client:[2025-09-03T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-09-28 10:16:07,309 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:ecmwf.datastores.legacy_client:[2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


CDS client OK


In [ ]:
# download_era5land_chunked_full_parallel_by_var.py —— 在你的基础上：按“变量”并发
# -*- coding: utf-8 -*-
import argparse
import time
import random
from pathlib import Path
from typing import List, Tuple
import sys
from concurrent.futures import ThreadPoolExecutor, as_completed

import cdsapi

DATASET = "reanalysis-era5-land"

# ---------- 时间维度 ----------
ALL_DAYS: List[str] = [f"{d:02d}" for d in range(1, 32)]
ALL_HOURS: List[str] = [f"{h:02d}:00" for h in range(0, 24)]
ALL_MONTHS: List[str] = [f"{m:02d}" for m in range(1, 13)]

# ---------- 重试设置 ----------
MAX_RETRIES = 8
BASE_SLEEP = 10  # seconds

# ---------- 变量全集（你的清单 + 注释项全部纳入） ----------
VARIABLES: List[str] = [
    # 2m/skin/soil/lake temps
    "2m_dewpoint_temperature",
    "2m_temperature",
    "skin_temperature",
    "soil_temperature_level_1",
    "soil_temperature_level_2",
    "soil_temperature_level_3",
    "soil_temperature_level_4",
    "lake_bottom_temperature",
    "lake_ice_depth",
    "lake_ice_temperature",
    "lake_mix_layer_depth",
    "lake_mix_layer_temperature",
    "lake_shape_factor",
    # 你原注释掉的项（已启用）
    "lake_total_layer_temperature",
    "snow_albedo",
    "snow_cover",
    "snow_density",
    "snow_depth",
    "snow_depth_water_equivalent",
    "snowfall",
    "snowmelt",
    "temperature_of_snow_layer",
    "forecast_albedo",
    "surface_latent_heat_flux",
    "surface_net_solar_radiation",
    "surface_net_thermal_radiation",
    "surface_sensible_heat_flux",
    "surface_solar_radiation_downwards",
    "surface_thermal_radiation_downwards",
    "evaporation_from_bare_soil",
    "evaporation_from_open_water_surfaces_excluding_oceans",
    "evaporation_from_the_top_of_canopy",
    "evaporation_from_vegetation_transpiration",
    "potential_evaporation",
    "runoff",
    "snow_evaporation",
    "sub_surface_runoff",
    "surface_runoff",
    "total_evaporation",
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "surface_pressure",
    "total_precipitation",
    "leaf_area_index_high_vegetation",
    "leaf_area_index_low_vegetation",
    "high_vegetation_cover",
    "glacier_mask",
    "lake_cover",
    "low_vegetation_cover",
    "lake_total_depth",
    "land_sea_mask",
    "soil_type",
    "type_of_high_vegetation",
    "type_of_low_vegetation",
]

def build_request(
    variable: str,
    year: str,
    month: str,
    area_box: Tuple[float, float, float, float],
    fmt: str,
) -> dict:
    north, west, south, east = area_box
    req = {
        "variable": variable,
        "year": year,
        "month": month,
        "day": ALL_DAYS,
        "time": ALL_HOURS,
        "area": [north, west, south, east],  # N W S E
        "format": fmt,                       # "grib" | "netcdf"
        "download_format": "zip",
        # "product_type": "reanalysis",
    }
    return req

def safe_retrieve(client: cdsapi.Client, dataset: str, request: dict, target_path: Path):
    """下载单个分块（含指数退避 + 轻度抖动），返回 True/False"""
    # 轻度抖动，降低“羊群效应”
    time.sleep(random.uniform(0.3, 1.0))
    attempt = 0
    while True:
        try:
            client.retrieve(dataset, request).download(str(target_path))
            return True
        except Exception as e:
            attempt += 1
            msg = str(e).lower()
            # 明确不可恢复的错误（变量无效/不可用/无数据）直接跳过
            unrecoverable_signals = [
                "unavailable",
                "not available",
                "invalid",
                "does not match",
                "no data",
                "bad request",
                "cannot be found",
            ]
            if any(s in msg for s in unrecoverable_signals):
                print(f"[ERROR] Unrecoverable for {target_path.name}: {e}")
                return False

            if attempt > MAX_RETRIES:
                print(f"[ERROR] Max retries exceeded for {target_path.name}: {e}")
                return False

            sleep_s = BASE_SLEEP * (2 ** (attempt - 1)) * random.uniform(0.85, 1.15)
            print(f"[WARN] Download failed (attempt {attempt}/{MAX_RETRIES}): {e}")
            print(f"       Sleeping {sleep_s:.0f}s then retrying...")
            time.sleep(sleep_s)

def parse_args_with_defaults():
    parser = argparse.ArgumentParser(
        description="ERA5-Land downloader (split by variable × year × month), parallel by VARIABLE"
    )
    # —— 给出默认值，不再强制要求 —— #
    parser.add_argument("--out_dir", type=str, default="./era5land",
                        help="输出根目录（默认 ./era5land）")
    parser.add_argument("--bbox", nargs=4, type=float,
                        default=[60.86, -6.23, 49.86, 1.75],
                        metavar=("NORTH", "WEST", "SOUTH", "EAST"),
                        help="经纬度范围：N W S E（默认 60.86 -6.23 49.86 1.75）")
    parser.add_argument("--format", default="grib", choices=["grib", "netcdf"],
                        help="文件格式（默认 grib）")
    parser.add_argument("--years", nargs="+",
                        default=[str(y) for y in range(1997, 2023)],  # 1997–2022
                        help="年份列表（默认 1997..2022）")
    parser.add_argument("--months", nargs="+", default=ALL_MONTHS,
                        help="月份列表（默认 01..12）")
    parser.add_argument("--variables", nargs="+", default=VARIABLES,
                        help="变量名列表（默认为脚本内置全集）")
    parser.add_argument("--skip_existing", action="store_true",
                        help="若目标文件已存在则跳过")
    parser.add_argument("--max_workers", type=int, default=3,
                        help="并发的变量数（建议 2–3）")
    # 如果在 Notebook 中直接运行，且没有传任何参数，也能用默认值
    try:
        return parser.parse_args([])
    except SystemExit:
        # 在某些环境 parse_args([]) 会触发 SystemExit，退回到标准方式
        return parser.parse_args()

def download_one_variable(var: str, args) -> tuple[str, int, int]:
    """在一个线程内：顺序下载某个变量的所有 年×月，返回 (var, ok, fail)"""
    client = cdsapi.Client()  # 每个线程各自的 client
    ok = 0
    fail = 0

    out_root = Path(args.out_dir)

    for year in args.years:
        for month in args.months:
            subdir = out_root / var / str(year)
            subdir.mkdir(parents=True, exist_ok=True)

            suffix = "grib" if args.format == "grib" else "nc"
            target_name = f"{DATASET}_{var}_{year}-{month}.{suffix}.zip"
            target_path = subdir / target_name

            if args.skip_existing and target_path.exists():
                # 已存在直接视为成功，便于断点续跑
                # 你也可以换成校验 zip 完整性的逻辑
                continue_ok = True
                if continue_ok:
                    ok += 1
                    continue

            req = build_request(
                variable=var,
                year=str(year),
                month=f"{int(month):02d}",
                area_box=tuple(args.bbox),
                fmt=args.format,
            )

            print(f"[INFO][{var}] Downloading  {year}-{month}  -> {target_path}")
            success = safe_retrieve(client, DATASET, req, target_path)
            if success:
                ok += 1
            else:
                fail += 1

    return var, ok, fail

def main():
    # 在 Notebook/Colab 里，这里会采用默认值；命令行下可用参数覆盖
    if "ipykernel" in sys.modules or "google.colab" in sys.modules:
        args = parse_args_with_defaults()
    else:
        args = parse_args_with_defaults()

    variables = list(args.variables)
    if not variables:
        print("[WARN] 未提供变量列表，使用内置 VARIABLES。")
        variables = VARIABLES

    # 并发数量不超过变量数
    max_workers = max(1, min(args.max_workers, len(variables)))

    print(f"[INFO] Variables: {len(variables)} | Years: {len(args.years)} | Months: {len(args.months)}")
    print(f"[INFO] Parallel by VARIABLE with max_workers = {max_workers}")
    start = time.time()

    total_ok = 0
    total_fail = 0
    results = []

    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = {ex.submit(download_one_variable, var, args): var for var in variables}
        for fut in as_completed(futures):
            var, ok, fail = fut.result()
            results.append((var, ok, fail))
            total_ok += ok
            total_fail += fail
            print(f"[DONE][{var}] Ok={ok}, Fail={fail}")

    elapsed = time.time() - start
    print("\n================ SUMMARY ================")
    for var, ok, fail in sorted(results):
        print(f"{var:40s}  Ok={ok:4d}  Fail={fail:3d}")
    print(f"-----------------------------------------")
    print(f"TOTAL  Ok={total_ok}  Fail={total_fail}  | Elapsed: {elapsed/60:.1f} min")
    print("=========================================\n")

if __name__ == "__main__":
    main()


[INFO] Variables: 54 | Years: 26 | Months: 12
[INFO] Parallel by VARIABLE with max_workers = 3


2025-09-28 10:12:52,047 INFO [2025-09-03T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
INFO:ecmwf.datastores.legacy_client:[2025-09-03T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-09-28 10:12:52,049 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:ecmwf.datastores.legacy_client:[2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-28 10:12:52,245 INFO [2025-09-03T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
INFO:ecmwf.datastores.legacy_client:[2025-09-03T00:00:00] To improve our C3S service, we need to h

[INFO][skin_temperature] Downloading  1997-01  -> era5land/skin_temperature/1997/reanalysis-era5-land_skin_temperature_1997-01.grib.zip
[INFO]_temperature] Downloading  1997-01  -> era5land/2m_temperature/1997/reanalysis-era5-land_2m_temperature_1997-01.grib.zip


2025-09-28 10:12:52,255 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:ecmwf.datastores.legacy_client:[2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


[INFO]_dewpoint_temperature] Downloading  1997-01  -> era5land/2m_dewpoint_temperature/1997/reanalysis-era5-land_2m_dewpoint_temperature_1997-01.grib.zip


...

[INFO]_dewpoint_temperature] Downloading  1997-02  -> era5land/2m_dewpoint_temperature/1997/reanalysis-era5-land_2m_dewpoint_temperature_1997-02.grib.zip


2025-09-28 10:13:11,084 INFO Request ID is c96d1a54-2476-40f3-ba1c-e360b6529ef7
INFO:ecmwf.datastores.legacy_client:Request ID is c96d1a54-2476-40f3-ba1c-e360b6529ef7
2025-09-28 10:13:11,231 INFO status has been updated to accepted
INFO:ecmwf.datastores.legacy_client:status has been updated to accepted
2025-09-28 10:13:15,343 INFO status has been updated to running
INFO:ecmwf.datastores.legacy_client:status has been updated to running
2025-09-28 10:14:48,504 INFO status has been updated to successful
INFO:ecmwf.datastores.legacy_client:status has been updated to successful


76d9dacc7921e0bd06d9383480d75e62.zip:   0%|          | 0.00/4.47M [00:00<?, ?B/s]

[INFO]_temperature] Downloading  1997-02  -> era5land/2m_temperature/1997/reanalysis-era5-land_2m_temperature_1997-02.grib.zip


2025-09-28 10:14:51,964 INFO Request ID is f4c1333f-a60a-47f8-84fc-e9e2d4555026
INFO:ecmwf.datastores.legacy_client:Request ID is f4c1333f-a60a-47f8-84fc-e9e2d4555026
2025-09-28 10:14:52,108 INFO status has been updated to accepted
INFO:ecmwf.datastores.legacy_client:status has been updated to accepted


In [ ]:
# download_era5land_chunked_full_month_parallel.py —— 按“月份”并发（每年最多12并发）
# -*- coding: utf-8 -*-
import argparse
import time
import random
from pathlib import Path
from typing import List, Tuple
import sys
from concurrent.futures import ThreadPoolExecutor, as_completed

import cdsapi

DATASET = "reanalysis-era5-land"

# ---------- 时间维度 ----------
ALL_DAYS: List[str] = [f"{d:02d}" for d in range(1, 32)]
ALL_HOURS: List[str] = [f"{h:02d}:00" for h in range(0, 24)]
ALL_MONTHS: List[str] = [f"{m:02d}" for m in range(1, 13)]

# ---------- 重试设置 ----------
MAX_RETRIES = 8
BASE_SLEEP = 10  # seconds

# ---------- 变量全集（你的清单 + 注释项全部纳入） ----------
VARIABLES: List[str] = [
    # 2m/skin/soil/lake temps
    "2m_dewpoint_temperature",
    "2m_temperature",
    "skin_temperature",
    "soil_temperature_level_1",
    "soil_temperature_level_2",
    "soil_temperature_level_3",
    "soil_temperature_level_4",
    "lake_bottom_temperature",
    "lake_ice_depth",
    "lake_ice_temperature",
    "lake_mix_layer_depth",
    "lake_mix_layer_temperature",
    "lake_shape_factor",
    # 你原注释掉的项（已启用）
    "lake_total_layer_temperature",
    "snow_albedo",
    "snow_cover",
    "snow_density",
    "snow_depth",
    "snow_depth_water_equivalent",
    "snowfall",
    "snowmelt",
    "temperature_of_snow_layer",
    "forecast_albedo",
    "surface_latent_heat_flux",
    "surface_net_solar_radiation",
    "surface_net_thermal_radiation",
    "surface_sensible_heat_flux",
    "surface_solar_radiation_downwards",
    "surface_thermal_radiation_downwards",
    "evaporation_from_bare_soil",
    "evaporation_from_open_water_surfaces_excluding_oceans",
    "evaporation_from_the_top_of_canopy",
    "evaporation_from_vegetation_transpiration",
    "potential_evaporation",
    "runoff",
    "snow_evaporation",
    "sub_surface_runoff",
    "surface_runoff",
    "total_evaporation",
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "surface_pressure",
    "total_precipitation",
    "leaf_area_index_high_vegetation",
    "leaf_area_index_low_vegetation",
    "high_vegetation_cover",
    "glacier_mask",
    "lake_cover",
    "low_vegetation_cover",
    "lake_total_depth",
    "land_sea_mask",
    "soil_type",
    "type_of_high_vegetation",
    "type_of_low_vegetation",
]

def build_request(
    variable: str,
    year: str,
    month: str,
    area_box: Tuple[float, float, float, float],
    fmt: str,
) -> dict:
    north, west, south, east = area_box
    req = {
        "variable": variable,
        "year": year,
        "month": month,
        "day": ALL_DAYS,
        "time": ALL_HOURS,
        "area": [north, west, south, east],  # N W S E
        "format": fmt,                       # "grib" | "netcdf"
        "download_format": "zip",
        # "product_type": "reanalysis",
    }
    return req

def safe_retrieve(client: cdsapi.Client, dataset: str, request: dict, target_path: Path):
    attempt = 0
    # 轻微抖动，错峰请求
    time.sleep(random.uniform(0.3, 1.0))
    while True:
        try:
            client.retrieve(dataset, request).download(str(target_path))
            return True
        except Exception as e:
            attempt += 1
            msg = str(e).lower()
            # 明确不可恢复的错误（变量无效/不可用/无数据）直接跳过
            unrecoverable_signals = [
                "unavailable",
                "not available",
                "invalid",
                "does not match",
                "no data",
                "bad request",
                "cannot be found",
            ]
            if any(s in msg for s in unrecoverable_signals):
                print(f"[ERROR] Unrecoverable for {target_path.name}: {e}")
                return False

            if attempt > MAX_RETRIES:
                print(f"[ERROR] Max retries exceeded for {target_path.name}: {e}")
                return False

            sleep_s = BASE_SLEEP * (2 ** (attempt - 1)) * random.uniform(0.85, 1.15)
            print(f"[WARN] Download failed (attempt {attempt}/{MAX_RETRIES}): {e}")
            print(f"       Sleeping {sleep_s:.0f}s then retrying...")
            time.sleep(sleep_s)

def parse_args_with_defaults():
    parser = argparse.ArgumentParser(
        description="ERA5-Land downloader (split by variable × year × month) — month-level parallelism"
    )
    # —— 给出默认值，不再强制要求 —— #
    parser.add_argument("--out_dir", type=str, default="./era5land",
                        help="输出根目录（默认 ./era5land）")
    parser.add_argument("--bbox", nargs=4, type=float,
                        default=[60.86, -6.23, 49.86, 1.75],
                        metavar=("NORTH", "WEST", "SOUTH", "EAST"),
                        help="经纬度范围：N W S E（默认 60.86 -6.23 49.86 1.75）")
    parser.add_argument("--format", default="grib", choices=["grib", "netcdf"],
                        help="文件格式（默认 grib）")
    parser.add_argument("--years", nargs="+",
                        default=[str(y) for y in range(1997, 2023)],  # 1997–2022
                        help="年份列表（默认 1997..2022）")
    parser.add_argument("--months", nargs="+", default=ALL_MONTHS,
                        help="月份列表（默认 01..12）")
    parser.add_argument("--variables", nargs="+", default=VARIABLES,
                        help="变量名列表（默认为脚本内置全集）")
    parser.add_argument("--skip_existing", action="store_true",
                        help="若目标文件已存在则跳过")
    parser.add_argument("--month_workers", type=int, default=12,
                        help="每个 年×变量 的月份并发数（默认 12）")
    # 如果在 Notebook 中直接运行，且没有传任何参数，也能用默认值
    try:
        return parser.parse_args([])
    except SystemExit:
        # 在某些环境 parse_args([]) 会触发 SystemExit，退回到标准方式
        return parser.parse_args()

def download_one_month(var: str, year: str, month: str, args) -> bool:
    """并发任务：下载单个 变量×年×月 分块"""
    subdir = Path(args.out_dir) / var / str(year)
    subdir.mkdir(parents=True, exist_ok=True)

    suffix = "grib" if args.format == "grib" else "nc"
    target_name = f"{DATASET}_{var}_{year}-{month}.{suffix}.zip"
    target_path = subdir / target_name

    if args.skip_existing and target_path.exists():
        # 已存在直接视为成功（简易断点续跑）
        return True

    req = build_request(
        variable=var,
        year=str(year),
        month=f"{int(month):02d}",
        area_box=tuple(args.bbox),
        fmt=args.format,
    )

    print(f"[INFO][{var}][{year}] Downloading month={month} -> {target_path}")
    # 为了线程安全，这里每个任务各自实例化 client
    client = cdsapi.Client()
    return safe_retrieve(client, DATASET, req, target_path)

def main():
    # 在 Notebook/Colab 里，这里会采用默认值；命令行下可用参数覆盖
    if "ipykernel" in sys.modules or "google.colab" in sys.modules:
        args = parse_args_with_defaults()
    else:
        args = parse_args_with_defaults()

    total_ok = 0
    total_fail = 0

    for var in args.variables:
        for year in args.years:
            months = list(args.months)
            max_workers = max(1, min(args.month_workers, len(months)))
            print(f"\n[GROUP] var={var} year={year} | months={months} | month_workers={max_workers}")

            futures = []
            with ThreadPoolExecutor(max_workers=max_workers) as ex:
                for month in months:
                    futures.append(ex.submit(download_one_month, var, year, month, args))
                for fut in as_completed(futures):
                    ok = fut.result()
                    if ok: total_ok += 1
                    else:  total_fail += 1

    print(f"\n[DONE] Finished. Success: {total_ok}, Failed: {total_fail}")

if __name__ == "__main__":
    main()



[GROUP] var=2m_dewpoint_temperature year=1997 | months=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'] | month_workers=12
[INFO]_dewpoint_temperature][1997] Downloading month=01 -> era5land/2m_dewpoint_temperature/1997/reanalysis-era5-land_2m_dewpoint_temperature_1997-01.grib.zip
[INFO]_dewpoint_temperature][1997] Downloading month=02 -> era5land/2m_dewpoint_temperature/1997/reanalysis-era5-land_2m_dewpoint_temperature_1997-02.grib.zip
[INFO]_dewpoint_temperature][1997] Downloading month=03 -> era5land/2m_dewpoint_temperature/1997/reanalysis-era5-land_2m_dewpoint_temperature_1997-03.grib.zip
[INFO]_dewpoint_temperature][1997] Downloading month=05 -> era5land/2m_dewpoint_temperature/1997/reanalysis-era5-land_2m_dewpoint_temperature_1997-05.grib.zip
[INFO]_dewpoint_temperature][1997] Downloading month=04 -> era5land/2m_dewpoint_temperature/1997/reanalysis-era5-land_2m_dewpoint_temperature_1997-04.grib.zip
[INFO]_dewpoint_temperature][1997] Downloading month=06 -

2025-09-28 10:16:14,916 INFO [2025-09-03T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
INFO:ecmwf.datastores.legacy_client:[2025-09-03T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-09-28 10:16:14,917 INFO [2025-09-03T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-09-28 10:16:14,919 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:ecmwf.datastores.legacy_client:[2025-09-03T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
INFO:ecmwf.datastores.legacy_client:[2024-09-26